In [1]:
# Attempt to plot Khumbu dataset
%matplotlib notebook

In [2]:
# Import modules - make sure you activated your rasterenv environment!
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from pygeotools.lib import iolib, warplib, geolib, timelib, malib

#### DEM sources:
32 m DEM derived from WorldView Stereo imagery acquired from 2003 to 2016

In [5]:
# Input DEM filenames
dem_first_fn = '../data/khumbu_DEM_32m/20030507_0445_1010010001E01600_1010010001E01500-DEM_32m.tif'
dem_last_fn = '../data/khumbu_DEM_32m/20161105_0448_1050010007140900_1050010007140B00-DEM_32m.tif'

dem_fn_list = [dem_first_fn, dem_last_fn]

# Next step: Find a way to read in all the files based on ending ""

In [7]:
ds_list = warplib.memwarp_multi_fn(dem_fn_list, extent='intersection', res='min', t_srs=dem_last_fn)


Warping all inputs to the following:
Resolution: 32.0
Extent: [482608.0, 3092432.0, 487696.0, 3104208.0]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: cubic

1 of 2: ../data/khumbu_DEM_32m/20030507_0445_1010010001E01600_1010010001E01500-DEM_32m.tif
nl: 368 ns: 159 res: 32.000
2 of 2: ../data/khumbu_DEM_32m/20161105_0448_1050010007140900_1050010007140B00-DEM_32m.tif
nl: 368 ns: 159 res: 32.000


In [8]:
# Load datasets to NumPy masked arrays
dem_2003, dem_2016 = [iolib.ds_getma(i) for i in ds_list]
dem_list = [dem_2003, dem_2016]

In [9]:
print(dem_2003.shape)
print(dem_2003.dtype)

(368, 159)
float32


In [11]:
dem_2003 # check out that the masked values were noted in a mask layer from the iolib thing

masked_array(
  data=[[--, --, --, ..., 5477.2353515625, 5473.408203125,
         5467.373046875],
        [--, --, --, ..., 5467.6689453125, 5471.5732421875,
         5470.2099609375],
        [--, --, --, ..., 5459.9326171875, 5461.177734375, 5467.83984375],
        ...,
        [--, --, 4945.369140625, ..., 5594.8701171875, 5596.70703125,
         5604.021484375],
        [--, --, 4941.4912109375, ..., 5597.6181640625, 5604.826171875,
         5614.9873046875],
        [--, --, 4956.10546875, ..., 5590.6572265625, 5603.029296875,
         5613.73828125]],
  mask=[[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [ True,  True, False, ..., False, False, False],
        [ True,  True, False, ..., False, False, False],
        [ True,  True, False, ..., False, False, False]],
  fill_value=0.0,
  dtype=float32)

In [ ]:
# generate 3 panel plot for input arrays
def plot3panel(dem_list, clim=none, titles=None, cmap=)